### Cheat Detection System 👁️

In [1]:
import cv2
import dlib
import os
import numpy as np
import requests
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import pygame  # لتشغيل الصوت

# إعدادات البريد الإلكتروني
sender_email = "ramezkmal3@gmail.com"
sender_password = "gtau zgbk wvkz dxjn"
receiver_email = "mooka8888mooka@gmail.com"

# روابط التنزيل التلقائي
SHAPE_PREDICTOR_URL = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
SHAPE_PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
ALERT_SOUND_URL = "https://freesound.org/data/previews/669/669669.wav"  # صوت إنذار مضمون
ALERT_SOUND_PATH = "alert_sound.wav"

# مجلد لحفظ الصور عند اكتشاف الغش
os.makedirs("alerts", exist_ok=True)

# تنزيل ملف الشكل (Dlib)
def download_shape_predictor():
    if not os.path.exists(SHAPE_PREDICTOR_PATH):
        print("[+] جارٍ تنزيل ملف الوجه (68 نقطة)...")
        import bz2
        response = requests.get(SHAPE_PREDICTOR_URL, stream=True)
        with open(SHAPE_PREDICTOR_PATH + ".bz2", "wb") as f:
            for data in response.iter_content(1024):
                f.write(data)
        with bz2.BZ2File(SHAPE_PREDICTOR_PATH + ".bz2") as fr:
            with open(SHAPE_PREDICTOR_PATH, "wb") as fw:
                fw.write(fr.read())
        print("[+] تم تنزيل ملف الشكل بنجاح!")

# تنزيل صوت الإنذار
def download_alert_sound():
    if not os.path.exists(ALERT_SOUND_PATH):
        print("[+] جارٍ تنزيل صوت الإنذار...")
        r = requests.get(ALERT_SOUND_URL)
        with open(ALERT_SOUND_PATH, "wb") as f:
            f.write(r.content)
        print("[+] تم تنزيل صوت الإنذار!")

# حساب نسبة فتح العين (EAR)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)

# إرسال بريد إلكتروني مع الصورة
def send_email_alert(image_path, alert_type):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = "🚨 [تنبيه غش] اكتشاف سلوك مشكوك فيه"

    body = f"""
    ✅ تم اكتشاف سلوك مشكوك فيه:

    🔍 نوع الغش: {alert_type}
    ⏰ الوقت: {time.ctime()}
    
    📸 الرجاء مراجعة الصورة المرفقة لمزيد من التفاصيل.

    مع خالص التقدير،
    نظام مراقبة الامتحانات الذكي 🤖
    """
    msg.attach(MIMEText(body, 'plain'))

    # إرفاق الصورة
    if os.path.exists(image_path):
        with open(image_path, 'rb') as f:
            img_data = f.read()
        image = MIMEImage(img_data, name=os.path.basename(image_path))
        msg.attach(image)

    # إرسال البريد
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        print("[+] ✉️ تم إرسال البريد الإلكتروني!")
    except Exception as e:
        print("[-] ❌ خطأ في إرسال البريد:", e)

# تشغيل صوت الإنذار
def play_alert_sound():
    try:
        pygame.mixer.music.load(ALERT_SOUND_PATH)
        pygame.mixer.music.play()
    except Exception as e:
        print("[-] ❌ خطأ في تشغيل الصوت:", e)

# تنزيل الملفات المطلوبة
download_shape_predictor()
download_alert_sound()

# تهيئة pygame.mixer
pygame.mixer.init()

# تحميل النماذج
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(SHAPE_PREDICTOR_PATH)

# تهيئة الكاميرا (رقم 1 دائمًا)
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("[-] ❌ لا يمكن فتح الكاميرا! تأكد من توصيل كاميرا خارجية.")
    exit(1)

print("[+] ✅ الكاميرا الخارجية تعمل!")
print("[+] 🚀 جارٍ تشغيل النظام...")

# إعداد EAR
(lStart, lEnd) = (42, 48)
(rStart, rEnd) = (36, 42)
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 30  # عدد الإطارات المتتالية لتكون إغماضًا
COUNTER = 0
ALERT_COOLDOWN = 10  # ثواني
last_alert_time = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("[-] ❌ خطأ في قراءة الفريم من الكاميرا!")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        shape = np.array([[p.x, p.y] for p in landmarks.parts()])

        left_eye = shape[lStart:lEnd]
        right_eye = shape[rStart:rEnd]

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # رسم نقاط الوجه
        for i, (x, y) in enumerate(shape):
            if i in list(range(36, 48)):  # العينين
                color = (0, 0, 255) if ear < EAR_THRESHOLD else (0, 255, 0)
            else:
                color = (255, 255, 255)
            cv2.circle(frame, (x, y), 1, color, -1)

        # الكشف عن إغماض العين لفترة طويلة
        if ear < EAR_THRESHOLD:
            COUNTER += 1
            if COUNTER >= EAR_CONSEC_FRAMES and (time.time() - last_alert_time) > ALERT_COOLDOWN:
                timestamp = time.strftime("%Y%m%d-%H%M%S")
                img_path = f"alerts/cheat_{timestamp}.jpg"
                cv2.imwrite(img_path, frame)
                print("[!] 🚨 الكشف عن إغماض العين!")
                send_email_alert(img_path, "إغماض العين لفترة طويلة")
                play_alert_sound()
                last_alert_time = time.time()
        else:
            COUNTER = 0

        # الكشف عن وجود أكثر من شخص
        if len(faces) > 1 and (time.time() - last_alert_time) > ALERT_COOLDOWN:
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            img_path = f"alerts/cheat_{timestamp}.jpg"
            cv2.imwrite(img_path, frame)
            print("[!] 🚨 الكشف عن أكثر من شخص!")
            send_email_alert(img_path, "وجود أكثر من شخص")
            play_alert_sound()
            last_alert_time = time.time()

    # عرض الفيديو
    cv2.imshow("Cheat Detection System 👁️", frame)

    # إنهاء البرنامج عند الضغط على 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
[+] جارٍ تنزيل صوت الإنذار...
[+] تم تنزيل صوت الإنذار!
[+] ✅ الكاميرا الخارجية تعمل!
[+] 🚀 جارٍ تشغيل النظام...
[!] 🚨 الكشف عن إغماض العين!
[+] ✉️ تم إرسال البريد الإلكتروني!
[-] ❌ خطأ في تشغيل الصوت: Unknown WAVE format


In [2]:
import cv2
import dlib
import os
import numpy as np
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import pygame  # لتشغيل الصوت

# إعدادات البريد الإلكتروني
sender_email = "ramezkmal3@gmail.com"
sender_password = "gtau zgbk wvkz dxjn"
receiver_email = "ramez_apkareno@hotmail.com"

# مسار الصوت المحلي
ALERT_SOUND_PATH = r"C:\Users\ramez\Desktop\delip\mark\alert_sound.mp3"

# نقطة الأنف (من بين 68 نقطة في الوجه)
NOSE_TIP_INDEX = 30  # النقطة 30 هي طرف الأنف

# مجلد لحفظ الصور عند اكتشاف الغش
os.makedirs("alerts", exist_ok=True)

# إعداد EAR (نسبة فتح العينين)
(lStart, lEnd) = (42, 48)
(rStart, rEnd) = (36, 42)
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 30
COUNTER_EYE = 0

# إعداد زوايا الرأس (Head Pose)
# 3D model points.
model_points = np.array([
    (0.0, 0.0, 0.0),             # Nose tip
    (0.0, -330.0, -65.0),        # Chin
    (-225.0, 170.0, -135.0),     # Left eye left corner
    (225.0, 170.0, -135.0),      # Right eye right corner
    (-150.0, -150.0, -125.0),    # Left Mouth corner
    (150.0, -150.0, -125.0)      # Right mouth corner
])

# إعداد المحاور الزمنية للكشف عن حركة الرأس
HEAD_YAW_THRESHOLD = 0.3   # دوران يسار/يمين
HEAD_PITCH_THRESHOLD = 0.3 # رفع/خفض الرأس
HEAD_ALERT_TIME = {
    "yaw": 4,   # 4 ثوانٍ لليمين أو اليسار
    "pitch": 5  # 5 ثوانٍ للنظر لأسفل
}
HEAD_ALERT_COUNTERS = {
    "yaw": 0,
    "pitch": 0
}
HEAD_LAST_ALERT = {
    "yaw": 0,
    "pitch": 0
}
HEAD_ALERT_COOLDOWN = 10  # 10 ثوانٍ قبل التنبيه مرة أخرى

# إرسال بريد إلكتروني مع الصورة
def send_email_alert(image_path, alert_type):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"🚨 [تنبيه غش] {alert_type}"

    body = f"""
    ✅ تم اكتشاف سلوك مشكوك فيه:

    🔍 نوع الغش: {alert_type}
    ⏰ الوقت: {time.ctime()}
    
    📸 الرجاء مراجعة الصورة المرفقة لمزيد من التفاصيل.

    مع خالص التقدير،
    نظام مراقبة الامتحانات الذكي 🤖
    """
    msg.attach(MIMEText(body, 'plain'))

    # إرفاق الصورة
    if os.path.exists(image_path):
        with open(image_path, 'rb') as f:
            img_data = f.read()
        image = MIMEImage(img_data, name=os.path.basename(image_path))
        msg.attach(image)

    # إرسال البريد
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        print("[+] ✉️ تم إرسال البريد الإلكتروني!")
    except Exception as e:
        print("[-] ❌ خطأ في إرسال البريد:", e)

# تشغيل صوت الإنذار
def play_alert_sound():
    try:
        pygame.mixer.music.load(ALERT_SOUND_PATH)
        pygame.mixer.music.play()
    except Exception as e:
        print("[-] ❌ خطأ في تشغيل الصوت:", e)

# حساب نسبة فتح العين (EAR)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)

# حساب زوايا الرأس (Pitch, Yaw, Roll)
def get_head_pose(shape, frame):
    image_size = frame.shape
    focal_length = image_size[1]
    center = (image_size[1]/2, image_size[0]/2)
    camera_matrix = np.array([[focal_length, 0, center[0]],
                              [0, focal_length, center[1]],
                              [0, 0, 1]], dtype="double")

    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

    shape_2d = np.array([[shape[i, 0], shape[i, 1]] for i in [30, 8, 36, 45, 48, 54]], dtype="double")
    
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, shape_2d, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_UPNP)

    if not success:
        return None

    # Get rotational matrix
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    pose_mat = cv2.hconcat((rotation_matrix, translation_vector))
    _, _, _, _, _, _, euler_angle = cv2.decomposeProjectionMatrix(pose_mat)

    pitch, yaw, roll = [angle[0] for angle in euler_angle]

    return {
        "pitch": pitch,
        "yaw": yaw,
        "roll": roll
    }

# تهيئة pygame.mixer
pygame.mixer.init()

# تحميل نموذج Dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# تهيئة الكاميرا (رقم 1 دائمًا)
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("[-] ❌ لا يمكن فتح الكاميرا! تأكد من توصيل كاميرا خارجية.")
    exit(1)

print("[+] ✅ الكاميرا الخارجية تعمل!")
print("[+] 🚀 جارٍ تشغيل النظام...")

while True:
    ret, frame = cap.read()
    if not ret:
        print("[-] ❌ خطأ في قراءة الفريم من الكاميرا!")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        shape = np.array([[p.x, p.y] for p in landmarks.parts()])

        # --- كشف العينين ---
        left_eye = shape[lStart:lEnd]
        right_eye = shape[rStart:rEnd]
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # رسم نقاط الوجه
        for i, (x, y) in enumerate(shape):
            if i in list(range(36, 48)):  # العينين
                color = (0, 0, 255) if ear < EAR_THRESHOLD else (0, 255, 0)
            else:
                color = (255, 255, 255)
            cv2.circle(frame, (x, y), 1, color, -1)

        # الكشف عن إغماض العين لفترة طويلة
        if ear < EAR_THRESHOLD:
            COUNTER_EYE += 1
            if COUNTER_EYE >= EAR_CONSEC_FRAMES and (time.time() - HEAD_LAST_ALERT["pitch"]) > HEAD_ALERT_COOLDOWN:
                timestamp = time.strftime("%Y%m%d-%H%M%S")
                img_path = f"alerts/cheat_{timestamp}.jpg"
                cv2.imwrite(img_path, frame)
                print("[!] 🚨 الكشف عن إغماض العين!")
                send_email_alert(img_path, "إغماض العين لفترة طويلة")
                play_alert_sound()
                HEAD_LAST_ALERT["pitch"] = time.time()
        else:
            COUNTER_EYE = 0

        # --- كشف حركة الرأس ---
        head_pose = get_head_pose(shape, frame)
        if head_pose is not None:
            yaw = abs(head_pose["yaw"])
            pitch = head_pose["pitch"]

            # الكشف عن دوران الرأس (يمين/يسار)
            if yaw > HEAD_YAW_THRESHOLD:
                HEAD_ALERT_COUNTERS["yaw"] += 1
                if HEAD_ALERT_COUNTERS["yaw"] >= HEAD_ALERT_TIME["yaw"] * 10 and (time.time() - HEAD_LAST_ALERT["yaw"]) > HEAD_ALERT_COOLDOWN:
                    timestamp = time.strftime("%Y%m%d-%H%M%S")
                    img_path = f"alerts/cheat_{timestamp}.jpg"
                    cv2.imwrite(img_path, frame)
                    print(f"[!] 🚨 الكشف عن دوران الرأس ({int(yaw)} درجة)!")
                    send_email_alert(img_path, f"دوران الرأس ({int(yaw)} درجة)")
                    play_alert_sound()
                    HEAD_LAST_ALERT["yaw"] = time.time()
                    HEAD_ALERT_COUNTERS["yaw"] = 0
            else:
                HEAD_ALERT_COUNTERS["yaw"] = 0

            # الكشف عن النظر للأسفل
            if pitch > HEAD_PITCH_THRESHOLD:
                HEAD_ALERT_COUNTERS["pitch"] += 1
                if HEAD_ALERT_COUNTERS["pitch"] >= HEAD_ALERT_TIME["pitch"] * 10 and (time.time() - HEAD_LAST_ALERT["pitch"]) > HEAD_ALERT_COOLDOWN:
                    timestamp = time.strftime("%Y%m%d-%H%M%S")
                    img_path = f"alerts/cheat_{timestamp}.jpg"
                    cv2.imwrite(img_path, frame)
                    print(f"[!] 🚨 الكشف عن النظر لأسفل ({int(pitch)} درجة)!")
                    send_email_alert(img_path, f"النظر لأسفل ({int(pitch)} درجة)")
                    play_alert_sound()
                    HEAD_LAST_ALERT["pitch"] = time.time()
                    HEAD_ALERT_COUNTERS["pitch"] = 0
            else:
                HEAD_ALERT_COUNTERS["pitch"] = 0

    # عرض الفيديو
    cv2.imshow("Cheat Detection System 👁️", frame)

    # إنهاء البرنامج عند الضغط على 'q'
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[+] ✅ الكاميرا الخارجية تعمل!
[+] 🚀 جارٍ تشغيل النظام...
[!] 🚨 الكشف عن دوران الرأس (5 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن دوران الرأس (38 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن دوران الرأس (4 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن النظر لأسفل (175 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن دوران الرأس (27 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن دوران الرأس (21 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!


In [4]:
import cv2
import dlib
import os
import numpy as np
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import pygame  # لتشغيل الصوت

# إعدادات البريد الإلكتروني
sender_email = "ramezkmal3@gmail.com"
sender_password = "gtau zgbk wvkz dxjn"
receiver_email = "ramez_apkareno@hotmail.com"

# مسار الصوت المحلي
ALERT_SOUND_PATH = r"C:\Users\ramez\Desktop\delip\mark\alert_sound.mp3"

# نقطة الأنف (من بين 68 نقطة في الوجه)
NOSE_TIP_INDEX = 30  # النقطة 30 هي طرف الأنف

# مجلد لحفظ الصور عند اكتشاف الغش
os.makedirs("alerts", exist_ok=True)

# إعداد EAR (نسبة فتح العينين)
(lStart, lEnd) = (42, 48)
(rStart, rEnd) = (36, 42)
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 30
COUNTER_EYE = 0

# إعداد زوايا الرأس (Head Pose)
# 3D model points.
model_points = np.array([
    (0.0, 0.0, 0.0),             # Nose tip
    (0.0, -330.0, -65.0),        # Chin
    (-225.0, 170.0, -135.0),     # Left eye left corner
    (225.0, 170.0, -135.0),      # Right eye right corner
    (-150.0, -150.0, -125.0),    # Left Mouth corner
    (150.0, -150.0, -125.0)      # Right mouth corner
])

# إعداد المحاور الزمنية للكشف عن حركة الرأس
HEAD_YAW_THRESHOLD = 30   # دوران يسار/يمين (30 درجة)
HEAD_PITCH_THRESHOLD = 30 # رفع/خفض الرأس
HEAD_ROLL_THRESHOLD = 20  # دوران الرأس جانبيًا
HEAD_ALERT_TIME = {
    "yaw": 5,   # 5 ثوانٍ لليمين أو اليسار
    "pitch": 5, # 5 ثوانٍ للنظر لأسفل
    "roll": 5   # 5 ثوانٍ لدوران الرأس
}
HEAD_ALERT_COUNTERS = {
    "yaw": 0,
    "pitch": 0,
    "roll": 0
}
HEAD_LAST_ALERT = {
    "yaw": 0,
    "pitch": 0,
    "roll": 0
}
HEAD_ALERT_COOLDOWN = 10  # 10 ثوانٍ قبل التنبيه مرة أخرى

# الكشف عن وجود أكثر من شخص
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
MULTI_FACE_COOLDOWN = 10
MULTI_FACE_LAST_ALERT = 0

# إرسال بريد إلكتروني مع الصورة
def send_email_alert(image_path, alert_type):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"🚨 [تنبيه غش] {alert_type}"

    body = f"""
    ✅ تم اكتشاف سلوك مشكوك فيه:

    🔍 نوع الغش: {alert_type}
    ⏰ الوقت: {time.ctime()}
    
    📸 الرجاء مراجعة الصورة المرفقة لمزيد من التفاصيل.

    مع خالص التقدير،
    نظام مراقبة الامتحانات الذكي 🤖
    """
    msg.attach(MIMEText(body, 'plain'))

    # إرفاق الصورة
    if os.path.exists(image_path):
        with open(image_path, 'rb') as f:
            img_data = f.read()
        image = MIMEImage(img_data, name=os.path.basename(image_path))
        msg.attach(image)

    # إرسال البريد
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        print("[+] ✉️ تم إرسال البريد الإلكتروني!")
    except Exception as e:
        print("[-] ❌ خطأ في إرسال البريد:", e)

# تشغيل صوت الإنذار
def play_alert_sound():
    try:
        pygame.mixer.music.load(ALERT_SOUND_PATH)
        pygame.mixer.music.play()
    except Exception as e:
        print("[-] ❌ خطأ في تشغيل الصوت:", e)

# حساب نسبة فتح العين (EAR)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)

# حساب زوايا الرأس (Pitch, Yaw, Roll)
def get_head_pose(shape, frame):
    image_size = frame.shape
    focal_length = image_size[1]
    center = (image_size[1]/2, image_size[0]/2)
    camera_matrix = np.array([[focal_length, 0, center[0]],
                              [0, focal_length, center[1]],
                              [0, 0, 1]], dtype="double")

    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

    shape_2d = np.array([[shape[i, 0], shape[i, 1]] for i in [30, 8, 36, 45, 48, 54]], dtype="double")
    
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, shape_2d, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_UPNP)

    if not success:
        return None

    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    pose_mat = cv2.hconcat((rotation_matrix, translation_vector))
    _, _, _, _, _, _, euler_angle = cv2.decomposeProjectionMatrix(pose_mat)

    pitch, yaw, roll = [angle[0] for angle in euler_angle]

    return {
        "pitch": pitch,
        "yaw": yaw,
        "roll": roll
    }

# تهيئة pygame.mixer
pygame.mixer.init()

# تحميل نموذج Dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# تهيئة الكاميرا (رقم 1 دائمًا)
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("[-] ❌ لا يمكن فتح الكاميرا! تأكد من توصيل كاميرا خارجية.")
    exit(1)

print("[+] ✅ الكاميرا الخارجية تعمل!")
print("[+] 🚀 جارٍ تشغيل النظام...")

while True:
    ret, frame = cap.read()
    if not ret:
        print("[-] ❌ خطأ في قراءة الفريم من الكاميرا!")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    # --- الكشف عن وجود أكثر من شخص ---
    faces_cv = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces_cv) > 1 and (time.time() - MULTI_FACE_LAST_ALERT) > HEAD_ALERT_COOLDOWN:
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        img_path = f"alerts/cheat_{timestamp}.jpg"
        cv2.imwrite(img_path, frame)
        print("[!] 🚨 الكشف عن وجود أكثر من شخص!")
        send_email_alert(img_path, "وجود أكثر من شخص")
        play_alert_sound()
        MULTI_FACE_LAST_ALERT = time.time()

    for face in faces:
        landmarks = predictor(gray, face)
        shape = np.array([[p.x, p.y] for p in landmarks.parts()])

        # --- كشف العينين ---
        left_eye = shape[lStart:lEnd]
        right_eye = shape[rStart:rEnd]
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # رسم نقاط الوجه
        for i, (x, y) in enumerate(shape):
            if i in list(range(36, 48)):  # العينين
                color = (0, 0, 255) if ear < EAR_THRESHOLD else (0, 255, 0)
            else:
                color = (255, 255, 255)
            cv2.circle(frame, (x, y), 1, color, -1)

        # الكشف عن إغماض العين لفترة طويلة
        if ear < EAR_THRESHOLD:
            COUNTER_EYE += 1
            if COUNTER_EYE >= EAR_CONSEC_FRAMES and (time.time() - HEAD_LAST_ALERT["pitch"]) > HEAD_ALERT_COOLDOWN:
                timestamp = time.strftime("%Y%m%d-%H%M%S")
                img_path = f"alerts/cheat_{timestamp}.jpg"
                cv2.imwrite(img_path, frame)
                print("[!] 🚨 الكشف عن إغماض العين!")
                send_email_alert(img_path, "إغماض العين لفترة طويلة")
                play_alert_sound()
                HEAD_LAST_ALERT["pitch"] = time.time()
        else:
            COUNTER_EYE = 0

        # --- كشف حركة الرأس ---
        head_pose = get_head_pose(shape, frame)
        if head_pose is not None:
            yaw = abs(head_pose["yaw"])
            pitch = abs(head_pose["pitch"])
            roll = abs(head_pose["roll"])

            # --- دوران الرأس (Yaw)
            if yaw > HEAD_YAW_THRESHOLD:
                HEAD_ALERT_COUNTERS["yaw"] += 1
                if HEAD_ALERT_COUNTERS["yaw"] >= HEAD_ALERT_TIME["yaw"] * 10 and (time.time() - HEAD_LAST_ALERT["yaw"]) > HEAD_ALERT_COOLDOWN:
                    timestamp = time.strftime("%Y%m%d-%H%M%S")
                    img_path = f"alerts/cheat_{timestamp}.jpg"
                    cv2.imwrite(img_path, frame)
                    print(f"[!] 🚨 الكشف عن دوران الرأس ({int(yaw)} درجة)!")
                    send_email_alert(img_path, f"دوران الرأس ({int(yaw)} درجة)")
                    play_alert_sound()
                    HEAD_LAST_ALERT["yaw"] = time.time()
                    HEAD_ALERT_COUNTERS["yaw"] = 0
            else:
                HEAD_ALERT_COUNTERS["yaw"] = 0

            # --- النظر للأسفل (Pitch)
            if pitch > HEAD_PITCH_THRESHOLD:
                HEAD_ALERT_COUNTERS["pitch"] += 1
                if HEAD_ALERT_COUNTERS["pitch"] >= HEAD_ALERT_TIME["pitch"] * 10 and (time.time() - HEAD_LAST_ALERT["pitch"]) > HEAD_ALERT_COOLDOWN:
                    timestamp = time.strftime("%Y%m%d-%H%M%S")
                    img_path = f"alerts/cheat_{timestamp}.jpg"
                    cv2.imwrite(img_path, frame)
                    print(f"[!] 🚨 الكشف عن النظر لأسفل ({int(pitch)} درجة)!")
                    send_email_alert(img_path, f"النظر لأسفل ({int(pitch)} درجة)")
                    play_alert_sound()
                    HEAD_LAST_ALERT["pitch"] = time.time()
                    HEAD_ALERT_COUNTERS["pitch"] = 0
            else:
                HEAD_ALERT_COUNTERS["pitch"] = 0

            # --- دوران الرأس جانبيًا (Roll)
            if roll > HEAD_ROLL_THRESHOLD:
                HEAD_ALERT_COUNTERS["roll"] += 1
                if HEAD_ALERT_COUNTERS["roll"] >= HEAD_ALERT_TIME["roll"] * 10 and (time.time() - HEAD_LAST_ALERT["roll"]) > HEAD_ALERT_COOLDOWN:
                    timestamp = time.strftime("%Y%m%d-%H%M%S")
                    img_path = f"alerts/cheat_{timestamp}.jpg"
                    cv2.imwrite(img_path, frame)
                    print(f"[!] 🚨 الكشف عن دوران الرأس جانبيًا ({int(roll)} درجة)!")
                    send_email_alert(img_path, f"دوران الرأس جانبيًا ({int(roll)} درجة)")
                    play_alert_sound()
                    HEAD_LAST_ALERT["roll"] = time.time()
                    HEAD_ALERT_COUNTERS["roll"] = 0
            else:
                HEAD_ALERT_COUNTERS["roll"] = 0

    # عرض الفيديو
    cv2.imshow("Cheat Detection System 👁️", frame)

    # إنهاء البرنامج عند الضغط على 'q'
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[+] ✅ الكاميرا الخارجية تعمل!
[+] 🚀 جارٍ تشغيل النظام...
[!] 🚨 الكشف عن إغماض العين!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن النظر لأسفل (177 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن النظر لأسفل (175 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن النظر لأسفل (172 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن وجود أكثر من شخص!
[+] ✉️ تم إرسال البريد الإلكتروني!
[!] 🚨 الكشف عن النظر لأسفل (178 درجة)!
[+] ✉️ تم إرسال البريد الإلكتروني!


### with mediapipe

In [8]:
import cv2
import mediapipe as mp
import numpy as np
import time
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import pygame
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import math

# Email settings
sender_email = "ramezkmal3@gmail.com"
sender_password = "gtau zgbk wvkz dxjn"
receiver_email = "marktakawy2008@gmail.com"

# Local sound path (update as needed)
ALERT_SOUND_PATH = r"C:\Users\ramez\Desktop\delip\mark\alert_sound.mp3"

# Folder to save alert images
os.makedirs("alerts", exist_ok=True)

# MediaPipe initialization
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=2,
    refine_landmarks=True,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

# EAR settings
LEFT_EYE_INDICES = [362, 385, 387, 263, 373, 380]
RIGHT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
EAR_THRESHOLD = 0.27
EAR_CONSEC_FRAMES = 20  # 2 seconds at ~10 FPS
COUNTER_EYE = 0
LAST_EYE_ALERT = 0
ALERT_COOLDOWN = 10

# Gaze detection settings
GAZE_THRESHOLD = 0.25
GAZE_CONSEC_FRAMES = 20
COUNTER_GAZE_LEFT = 0
COUNTER_GAZE_RIGHT = 0
LAST_GAZE_ALERT = 0

# Head pose settings
MODEL_POINTS = np.array([
    (0.0, 0.0, 0.0),
    (0.0, -330.0, -65.0),
    (-225.0, 170.0, -135.0),
    (225.0, 170.0, -135.0),
    (-150.0, -150.0, -125.0),
    (150.0, -150.0, -125.0)
])
HEAD_YAW_THRESHOLD = 25
HEAD_PITCH_THRESHOLD = 25
HEAD_ROLL_THRESHOLD = 15
HEAD_CONSEC_FRAMES = 20
HEAD_ALERT_COUNTERS = {"yaw": 0, "pitch": 0, "roll": 0}
HEAD_LAST_ALERT = {"yaw": 0, "pitch": 0, "roll": 0}

# Multiple face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
MULTI_FACE_CONSEC_FRAMES = 20
MULTI_FACE_COUNTER = 0
MULTI_FACE_LAST_ALERT = 0

# Send email alert with clean image
def send_email_alert(image, alert_type):
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    img_path = f"alerts/cheat_{timestamp}.jpg"
    cv2.imwrite(img_path, image)

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"🚨 [Cheating Alert] {alert_type}"

    body = f"""
    ✅ Suspicious behavior detected:

    🔍 Cheat Type: {alert_type}
    ⏰ Time: {time.ctime()}
    
    📸 Please review the attached image for details.

    Best regards,
    Smart Exam Monitoring System 🤖
    """
    msg.attach(MIMEText(body, 'plain'))

    if os.path.exists(img_path):
        with open(img_path, 'rb') as f:
            img_data = f.read()
        image = MIMEImage(img_data, name=os.path.basename(img_path))
        msg.attach(image)

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        print("[+] ✉️ Email sent successfully!")
    except Exception as e:
        print(f"[-] ❌ Email sending error: {e}")

# Play alert sound
def play_alert_sound():
    try:
        pygame.mixer.music.load(ALERT_SOUND_PATH)
        pygame.mixer.music.play()
    except Exception as e:
        print(f"[-] ❌ Sound playback error: {e}")

# Calculate EAR
def eye_aspect_ratio(eye_points, landmarks):
    eye = [(landmarks[i].x, landmarks[i].y) for i in eye_points]
    A = math.sqrt((eye[1][0] - eye[5][0])**2 + (eye[1][1] - eye[5][1])**2)
    B = math.sqrt((eye[2][0] - eye[4][0])**2 + (eye[2][1] - eye[4][1])**2)
    C = math.sqrt((eye[0][0] - eye[3][0])**2 + (eye[0][1] - eye[3][1])**2)
    return (A + B) / (2.0 * C)

# Estimate gaze direction
def estimate_gaze(landmarks):
    left_iris = (landmarks[473].x, landmarks[473].y)
    right_iris = (landmarks[468].x, landmarks[468].y)
    left_eye_center = ((landmarks[362].x + landmarks[263].x) / 2, (landmarks[362].y + landmarks[263].y) / 2)
    right_eye_center = ((landmarks[33].x + landmarks[133].x) / 2, (landmarks[33].y + landmarks[133].y) / 2)
    left_gaze_x = left_iris[0] - left_eye_center[0]
    right_gaze_x = right_iris[0] - right_eye_center[0]
    return (left_gaze_x + right_gaze_x) / 2

# Calculate head pose
def get_head_pose(landmarks, frame):
    image_size = frame.shape
    focal_length = image_size[1]
    center = (image_size[1]/2, image_size[0]/2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")
    dist_coeffs = np.zeros((4,1))
    indices = [1, 152, 263, 33, 287, 57]
    shape_2d = np.array([(landmarks[i].x * frame.shape[1], landmarks[i].y * frame.shape[0]) for i in indices], dtype="double")
    
    success, rotation_vector, translation_vector = cv2.solvePnP(MODEL_POINTS, shape_2d, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_UPNP)
    
    if not success:
        return None

    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    pose_mat = cv2.hconcat((rotation_matrix, translation_vector))
    _, _, _, _, _, _, euler_angle = cv2.decomposeProjectionMatrix(pose_mat)
    pitch, yaw, roll = [angle[0] for angle in euler_angle]
    return {"pitch": pitch, "yaw": yaw, "roll": roll}

# Main detection function
def start_detection():
    pygame.mixer.init()
    cap = cv2.VideoCapture(1)
    
    if not cap.isOpened():
        messagebox.showerror("Error", "Cannot open external camera! Please ensure it is connected.")
        return

    print("[+] ✅ External camera is working!")
    print("[+] 🚀 Starting the system...")

    global COUNTER_EYE, COUNTER_GAZE_LEFT, COUNTER_GAZE_RIGHT, HEAD_ALERT_COUNTERS
    global LAST_EYE_ALERT, LAST_GAZE_ALERT, HEAD_LAST_ALERT, MULTI_FACE_COUNTER, MULTI_FACE_LAST_ALERT

    while True:
        ret, frame = cap.read()
        if not ret:
            print("[-] ❌ Error reading frame from camera!")
            break

        # Keep a clean frame for email
        clean_frame = frame.copy()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Cheating flag for color change
        is_cheating = False

        # Multiple face detection
        faces_cv = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces_cv) > 1:
            MULTI_FACE_COUNTER += 1
            is_cheating = True
            if MULTI_FACE_COUNTER >= MULTI_FACE_CONSEC_FRAMES and (time.time() - MULTI_FACE_LAST_ALERT) > ALERT_COOLDOWN:
                send_email_alert(clean_frame, "Multiple Faces Detected")
                play_alert_sound()
                MULTI_FACE_LAST_ALERT = time.time()
                MULTI_FACE_COUNTER = 0
        else:
            MULTI_FACE_COUNTER = 0

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                landmarks = face_landmarks.landmark

                # Eye closure detection
                left_ear = eye_aspect_ratio(LEFT_EYE_INDICES, landmarks)
                right_ear = eye_aspect_ratio(RIGHT_EYE_INDICES, landmarks)
                ear = (left_ear + right_ear) / 2.0
                if ear < EAR_THRESHOLD:
                    COUNTER_EYE += 1
                    is_cheating = True
                    if COUNTER_EYE >= EAR_CONSEC_FRAMES and (time.time() - LAST_EYE_ALERT) > ALERT_COOLDOWN:
                        send_email_alert(clean_frame, "Prolonged Eye Closure")
                        play_alert_sound()
                        LAST_EYE_ALERT = time.time()
                        COUNTER_EYE = 0
                else:
                    COUNTER_EYE = 0

                # Gaze detection
                gaze_x = estimate_gaze(landmarks)
                if gaze_x > GAZE_THRESHOLD:
                    COUNTER_GAZE_RIGHT += 1
                    is_cheating = True
                    if COUNTER_GAZE_RIGHT >= GAZE_CONSEC_FRAMES and (time.time() - LAST_GAZE_ALERT) > ALERT_COOLDOWN:
                        send_email_alert(clean_frame, "Looking Right")
                        play_alert_sound()
                        LAST_GAZE_ALERT = time.time()
                        COUNTER_GAZE_RIGHT = 0
                elif gaze_x < -GAZE_THRESHOLD:
                    COUNTER_GAZE_LEFT += 1
                    is_cheating = True
                    if COUNTER_GAZE_LEFT >= GAZE_CONSEC_FRAMES and (time.time() - LAST_GAZE_ALERT) > ALERT_COOLDOWN:
                        send_email_alert(clean_frame, "Looking Left")
                        play_alert_sound()
                        LAST_GAZE_ALERT = time.time()
                        COUNTER_GAZE_LEFT = 0
                else:
                    COUNTER_GAZE_LEFT = 0
                    COUNTER_GAZE_RIGHT = 0

                # Head pose detection
                head_pose = get_head_pose(landmarks, frame)
                if head_pose:
                    yaw = abs(head_pose["yaw"])
                    pitch = abs(head_pose["pitch"])
                    roll = abs(head_pose["roll"])

                    if yaw > HEAD_YAW_THRESHOLD:
                        HEAD_ALERT_COUNTERS["yaw"] += 1
                        is_cheating = True
                        if HEAD_ALERT_COUNTERS["yaw"] >= HEAD_CONSEC_FRAMES and (time.time() - HEAD_LAST_ALERT["yaw"]) > ALERT_COOLDOWN:
                            send_email_alert(clean_frame, f"Head Yaw ({int(yaw)} degrees)")
                            play_alert_sound()
                            HEAD_LAST_ALERT["yaw"] = time.time()
                            HEAD_ALERT_COUNTERS["yaw"] = 0
                    else:
                        HEAD_ALERT_COUNTERS["yaw"] = 0

                    if pitch > HEAD_PITCH_THRESHOLD:
                        HEAD_ALERT_COUNTERS["pitch"] += 1
                        is_cheating = True
                        if HEAD_ALERT_COUNTERS["pitch"] >= HEAD_CONSEC_FRAMES and (time.time() - HEAD_LAST_ALERT["pitch"]) > ALERT_COOLDOWN:
                            send_email_alert(clean_frame, f"Looking Down ({int(pitch)} degrees)")
                            play_alert_sound()
                            HEAD_LAST_ALERT["pitch"] = time.time()
                            HEAD_ALERT_COUNTERS["pitch"] = 0
                    else:
                        HEAD_ALERT_COUNTERS["pitch"] = 0

                    if roll > HEAD_ROLL_THRESHOLD:
                        HEAD_ALERT_COUNTERS["roll"] += 1
                        is_cheating = True
                        if HEAD_ALERT_COUNTERS["roll"] >= HEAD_CONSEC_FRAMES and (time.time() - HEAD_LAST_ALERT["roll"]) > ALERT_COOLDOWN:
                            send_email_alert(clean_frame, f"Head Roll ({int(roll)} degrees)")
                            play_alert_sound()
                            HEAD_LAST_ALERT["roll"] = time.time()
                            HEAD_ALERT_COUNTERS["roll"] = 0
                    else:
                        HEAD_ALERT_COUNTERS["roll"] = 0

                # Draw landmarks with color based on cheating
                color = (0, 0, 255) if is_cheating else (0, 255, 0)
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,  # Only contours for speed
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=color, thickness=1),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=color, thickness=1)
                )

        cv2.imshow("Cheat Detection System 👁️", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    face_mesh.close()

# Tkinter GUI
def create_gui():
    root = tk.Tk()
    root.title("Smart Exam Monitoring System")
    root.geometry("600x400")
    root.configure(bg="#1e3a8a")

    welcome_label = tk.Label(
        root,
        text="Welcome to the Smart Exam Monitoring System! 🤖\nEnsure your camera is connected and press Start to begin.",
        font=("Helvetica", 14, "bold"),
        fg="#ffffff",
        bg="#1e3a8a",
        wraplength=500,
        justify="center"
    )
    welcome_label.pack(pady=50)

    start_button = tk.Button(
        root,
        text="Start Monitoring",
        font=("Helvetica", 16, "bold"),
        bg="#22c55e",
        fg="#ffffff",
        activebackground="#16a34a",
        activeforeground="#ffffff",
        command=lambda: [root.destroy(), start_detection()],
        width=20,
        height=2
    )
    start_button.pack(pady=20)

    footer_label = tk.Label(
        root,
        text="Powered by xAI",
        font=("Helvetica", 10),
        fg="#93c5fd",
        bg="#1e3a8a"
    )
    footer_label.pack(side="bottom", pady=10)

    root.mainloop()

if __name__ == "__main__":
    create_gui()

[+] ✅ External camera is working!
[+] 🚀 Starting the system...
[+] ✉️ Email sent successfully!
[+] ✉️ Email sent successfully!
[+] ✉️ Email sent successfully!
[+] ✉️ Email sent successfully!
[+] ✉️ Email sent successfully!
[-] ❌ Error reading frame from camera!
